In [1]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()


In [2]:
import tensorflow as tf
from tensorflow import keras
import scipy.io as sc
import numpy as np
from sklearn import preprocessing

# this function is used to transfer one column label to one hot label

# def one_hot(y_):
#     n_values = np.max(y_) + 1
#     return keras.utils.to_categorical(y_, num_classes=n_values)
feature = sc.loadmat("S1_nolabel6.mat")
all = feature['S1_nolabel6']

def one_hot(y_):
    # Function to encode output labels from number indexes
    # e.g.: [[5], [0], [3]] --> [[0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]
    y_ = y_.reshape(len(y_))
    n_values = np.max(y_) + 1
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]

print(all.shape)
np.random.shuffle(all)  # mix eeg_all

final = 2800 * 10
all = all[0:final]
feature_all = all[:, 0:64]
label = all[:, 64:65]

# z-score
feature_all=scaler.fit_transform(feature_all)
# feature_all = preprocessing.scale(feature_all)

no_fea = feature_all.shape[-1]
print("feature all",feature_all.shape)
print(no_fea)
label_all = one_hot(label)
print(label_all.shape)

n_classes = 6

(29738, 65)
feature all (28000, 64)
64
(28000, 6)


In [3]:
from tensorflow.keras.losses import Loss
from tensorflow.keras.regularizers import l2
from tensorflow.keras import losses


def custom_loss(alpha=0.001):
    # Define the custom loss function
    def loss_function(y_true, y_pred):
        # Calculate the softmax cross-entropy loss
        cross_entropy_loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)

        # cross_entropy_loss=losses.CategoricalCrossentropy()(y_true,y_pred)
        # Calculate the L2 loss
        l2_loss = alpha * tf.reduce_sum([tf.nn.l2_loss(var) for var in model.trainable_variables])

        # Combine the two losses
        total_loss = cross_entropy_loss + l2_loss

        return total_loss

    return loss_function

In [4]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2
import numpy as np
import time

print("cnn input feature shape", feature_all.shape)
n_fea = feature_all.shape[-1]
final = all.shape[0]
middle_number = int(final * 3 / 4)
feature_training = feature_all[0:middle_number]
feature_testing = feature_all[middle_number:final]
label_training = label_all[0:middle_number]
label_testing = label_all[middle_number:final]
label_ww = label_all[middle_number:final]  # for the confusion matrix
print("label_testing", label_testing.shape)
a = feature_training
b = feature_testing
print(feature_training.shape)
print(feature_testing.shape)

a=a.reshape(-1,1,n_fea,1)
b=b.reshape(-1,1,n_fea,1)
keep = 0.999999
batch_size = final - middle_number
n_group = 3

all_features = feature_all.reshape(-1, 1, n_fea, 1)
# Create a Sequential model

model = Sequential()

bias=tf.keras.initializers.Constant(0.1)
weight=tf.keras.initializers.TruncatedNormal(mean=0., stddev=0.1)

# Convolutional Layer 1

model.add(Conv2D(2, (1, 1),kernel_initializer=weight,strides=(1, 1),
bias_initializer=bias, activation='relu', input_shape=(1, n_fea, 1),padding="same"))


model.add(MaxPooling2D(pool_size=(1, 2),padding="same"))

# Flatten the output
model.add(Flatten())

# Fully Connected Layer 1
model.add(Dense(120, activation='sigmoid',kernel_initializer=weight,bias_initializer=bias))

# Dropout Layer
model.add(Dropout(0))

# Fully Connected Layer 2
model.add(Dense(n_classes,kernel_initializer=weight,bias_initializer=bias))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.04),
              loss=custom_loss(),
              metrics=['accuracy'])

print("Model Summary:")
model.summary()

# Training the model
# batch_size = final - middle_number
# epochs = 1
model.fit(a, label_training, epochs=1500, batch_size=batch_size)
# for step in range(epochs):
#     model.fit(a, label_training, epochs=1, batch_size=batch_size)

#     if step % 5 == 0:
#         cost = model.evaluate(b, label_testing, verbose=0)
#         acc_cnn_t = cost[1]
#         print(f'the step is: {step}, the acc is {acc_cnn_t}, the cost is {cost[0]}')

# Evaluate the model
acc_cnn = model.evaluate(b, label_testing)
print("Final Accuracy on Test Data:", acc_cnn[1])



cnn_up_to_dropout = keras.Model(inputs=model.inputs, outputs=model.layers[-2].output)
feature_all_cnn = cnn_up_to_dropout.predict(all_features)
print("----->")
print(feature_all_cnn.shape)

# Save the model
model.save('cnn_with_dropout.h5')

print("the shape of cnn output features", feature_all_cnn.shape, label_all.shape)

cnn input feature shape (28000, 64)
label_testing (7000, 6)
(21000, 64)
(7000, 64)
Model Summary:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1, 64, 2)          4         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 1, 32, 2)         0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 120)               7800      
                                                                 
 dropout (Dropout)           (None, 120)               0         
                                                                 
 dense_1 (Dense)        

D:\Anaconda\envs\tf2\lib\site-packages\keras\initializers\initializers_v2.py:121: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


3/3 [==============================] - 1s 25ms/step - loss: 2.3263 - accuracy: 0.2142
Epoch 2/1500
3/3 [==============================] - 0s 23ms/step - loss: 1.9210 - accuracy: 0.2185
Epoch 3/1500
3/3 [==============================] - 0s 23ms/step - loss: 1.8750 - accuracy: 0.2774
Epoch 4/1500
3/3 [==============================] - 0s 22ms/step - loss: 1.7002 - accuracy: 0.2412
Epoch 5/1500
3/3 [==============================] - 0s 24ms/step - loss: 1.7123 - accuracy: 0.2672
Epoch 6/1500
3/3 [==============================] - 0s 26ms/step - loss: 1.6291 - accuracy: 0.3380
Epoch 7/1500
3/3 [==============================] - 0s 25ms/step - loss: 1.6342 - accuracy: 0.3281
Epoch 8/1500
3/3 [==============================] - 0s 25ms/step - loss: 1.6067 - accuracy: 0.3383
Epoch 9/1500
3/3 [==============================] - 0s 26ms/step - loss: 1.5811 - accuracy: 0.3401
Epoch 10/1500
3/3 [==============================] - 0s 24ms/step - loss: 1.5704 - accuracy: 0.3532
Epoch 11/1500
3/3 [===

3/3 [==============================] - 0s 23ms/step - loss: 1.2805 - accuracy: 0.5330
Epoch 84/1500
3/3 [==============================] - 0s 23ms/step - loss: 1.2535 - accuracy: 0.5459
Epoch 85/1500
3/3 [==============================] - 0s 23ms/step - loss: 1.2412 - accuracy: 0.5408
Epoch 86/1500
3/3 [==============================] - 0s 22ms/step - loss: 1.2201 - accuracy: 0.5649
Epoch 87/1500
3/3 [==============================] - 0s 24ms/step - loss: 1.2054 - accuracy: 0.5675
Epoch 88/1500
3/3 [==============================] - 0s 22ms/step - loss: 1.2001 - accuracy: 0.5706
Epoch 89/1500
3/3 [==============================] - 0s 22ms/step - loss: 1.1837 - accuracy: 0.5823
Epoch 90/1500
3/3 [==============================] - 0s 24ms/step - loss: 1.1839 - accuracy: 0.5797
Epoch 91/1500
3/3 [==============================] - 0s 21ms/step - loss: 1.2103 - accuracy: 0.5657
Epoch 92/1500
3/3 [==============================] - 0s 22ms/step - loss: 1.2099 - accuracy: 0.5750
Epoch 93/1500


3/3 [==============================] - 0s 23ms/step - loss: 1.1067 - accuracy: 0.6409
Epoch 165/1500
3/3 [==============================] - 0s 24ms/step - loss: 1.1027 - accuracy: 0.6391
Epoch 166/1500
3/3 [==============================] - 0s 22ms/step - loss: 1.1368 - accuracy: 0.6230
Epoch 167/1500
3/3 [==============================] - 0s 24ms/step - loss: 1.1580 - accuracy: 0.6107
Epoch 168/1500
3/3 [==============================] - 0s 22ms/step - loss: 1.2236 - accuracy: 0.5726
Epoch 169/1500
3/3 [==============================] - 0s 22ms/step - loss: 1.1902 - accuracy: 0.5974
Epoch 170/1500
3/3 [==============================] - 0s 23ms/step - loss: 1.1956 - accuracy: 0.5925
Epoch 171/1500
3/3 [==============================] - 0s 23ms/step - loss: 1.1727 - accuracy: 0.6029
Epoch 172/1500
3/3 [==============================] - 0s 22ms/step - loss: 1.1646 - accuracy: 0.6088
Epoch 173/1500
3/3 [==============================] - 0s 23ms/step - loss: 1.1652 - accuracy: 0.6069
Epoch

...

3/3 [==============================] - 0s 24ms/step - loss: 0.9844 - accuracy: 0.7282
Epoch 1135/1500
3/3 [==============================] - 0s 29ms/step - loss: 0.9798 - accuracy: 0.7283
Epoch 1136/1500
3/3 [==============================] - 0s 24ms/step - loss: 0.9764 - accuracy: 0.7353
Epoch 1137/1500
3/3 [==============================] - 0s 25ms/step - loss: 0.9605 - accuracy: 0.7370
Epoch 1138/1500
3/3 [==============================] - 0s 24ms/step - loss: 0.9656 - accuracy: 0.7337
Epoch 1139/1500
3/3 [==============================] - 0s 24ms/step - loss: 0.9461 - accuracy: 0.7456
Epoch 1140/1500
3/3 [==============================] - 0s 25ms/step - loss: 0.9973 - accuracy: 0.7205
Epoch 1141/1500
3/3 [==============================] - 0s 23ms/step - loss: 0.9866 - accuracy: 0.7221
Epoch 1142/1500
3/3 [==============================] - 0s 23ms/step - loss: 1.0145 - accuracy: 0.7066
Epoch 1143/1500
3/3 [==============================] - 0s 23ms/step - loss: 1.0414 - accuracy: 0.6

3/3 [==============================] - 0s 22ms/step - loss: 0.9622 - accuracy: 0.7351
Epoch 1215/1500
3/3 [==============================] - 0s 23ms/step - loss: 0.9705 - accuracy: 0.7304
Epoch 1216/1500
3/3 [==============================] - 0s 23ms/step - loss: 0.9715 - accuracy: 0.7304
Epoch 1217/1500
3/3 [==============================] - 0s 23ms/step - loss: 0.9675 - accuracy: 0.7321
Epoch 1218/1500
3/3 [==============================] - 0s 23ms/step - loss: 0.9876 - accuracy: 0.7197
Epoch 1219/1500
3/3 [==============================] - 0s 24ms/step - loss: 0.9808 - accuracy: 0.7288
Epoch 1220/1500
3/3 [==============================] - 0s 24ms/step - loss: 0.9786 - accuracy: 0.7301
Epoch 1221/1500
3/3 [==============================] - 0s 25ms/step - loss: 1.0015 - accuracy: 0.7174
Epoch 1222/1500
3/3 [==============================] - 0s 22ms/step - loss: 1.0907 - accuracy: 0.6651
Epoch 1223/1500
3/3 [==============================] - 0s 23ms/step - loss: 1.1953 - accuracy: 0.6

3/3 [==============================] - 0s 23ms/step - loss: 1.1340 - accuracy: 0.6566
Epoch 1295/1500
3/3 [==============================] - 0s 24ms/step - loss: 1.1045 - accuracy: 0.6701
Epoch 1296/1500
3/3 [==============================] - 0s 23ms/step - loss: 1.0961 - accuracy: 0.6735
Epoch 1297/1500
3/3 [==============================] - 0s 23ms/step - loss: 1.0708 - accuracy: 0.6837
Epoch 1298/1500
3/3 [==============================] - 0s 27ms/step - loss: 1.0499 - accuracy: 0.6898
Epoch 1299/1500
3/3 [==============================] - 0s 26ms/step - loss: 1.0099 - accuracy: 0.7110
Epoch 1300/1500
3/3 [==============================] - 0s 23ms/step - loss: 1.0056 - accuracy: 0.7148
Epoch 1301/1500
3/3 [==============================] - 0s 23ms/step - loss: 0.9878 - accuracy: 0.7277
Epoch 1302/1500
3/3 [==============================] - 0s 23ms/step - loss: 0.9831 - accuracy: 0.7265
Epoch 1303/1500
3/3 [==============================] - 0s 22ms/step - loss: 0.9715 - accuracy: 0.7

3/3 [==============================] - 0s 25ms/step - loss: 1.0280 - accuracy: 0.7101
Epoch 1375/1500
3/3 [==============================] - 0s 23ms/step - loss: 1.0023 - accuracy: 0.7223
Epoch 1376/1500
3/3 [==============================] - 0s 24ms/step - loss: 0.9818 - accuracy: 0.7290
Epoch 1377/1500
3/3 [==============================] - 0s 23ms/step - loss: 1.0293 - accuracy: 0.7038
Epoch 1378/1500
3/3 [==============================] - 0s 26ms/step - loss: 0.9967 - accuracy: 0.7227
Epoch 1379/1500
3/3 [==============================] - 0s 26ms/step - loss: 0.9879 - accuracy: 0.7260
Epoch 1380/1500
3/3 [==============================] - 0s 23ms/step - loss: 1.0037 - accuracy: 0.7158
Epoch 1381/1500
3/3 [==============================] - 0s 23ms/step - loss: 0.9752 - accuracy: 0.7317
Epoch 1382/1500
3/3 [==============================] - 0s 23ms/step - loss: 1.0067 - accuracy: 0.7143
Epoch 1383/1500
3/3 [==============================] - 0s 24ms/step - loss: 1.0298 - accuracy: 0.6

3/3 [==============================] - 0s 25ms/step - loss: 1.0176 - accuracy: 0.7043
Epoch 1455/1500
3/3 [==============================] - 0s 24ms/step - loss: 0.9819 - accuracy: 0.7252
Epoch 1456/1500
3/3 [==============================] - 0s 23ms/step - loss: 0.9675 - accuracy: 0.7336
Epoch 1457/1500
3/3 [==============================] - 0s 23ms/step - loss: 1.0274 - accuracy: 0.7051
Epoch 1458/1500
3/3 [==============================] - 0s 23ms/step - loss: 1.0330 - accuracy: 0.7020
Epoch 1459/1500
3/3 [==============================] - 0s 23ms/step - loss: 1.0150 - accuracy: 0.7101
Epoch 1460/1500
3/3 [==============================] - 0s 23ms/step - loss: 1.0271 - accuracy: 0.7025
Epoch 1461/1500
3/3 [==============================] - 0s 23ms/step - loss: 1.0283 - accuracy: 0.7034
Epoch 1462/1500
3/3 [==============================] - 0s 23ms/step - loss: 1.0559 - accuracy: 0.6943
Epoch 1463/1500
3/3 [==============================] - 0s 24ms/step - loss: 1.0769 - accuracy: 0.6

In [5]:
acc_cnn = model.evaluate(b, label_testing)
print("Final Accuracy on Test Data:", acc_cnn[1])



cnn_up_to_dropout = keras.Model(inputs=model.inputs, outputs=model.layers[-2].output)
feature_all_cnn = cnn_up_to_dropout.predict(all_features)
print("----->")
print(feature_all_cnn.shape)

# Save the model
model.save('cnn_with_dropout.h5')

print("the shape of cnn output features", feature_all_cnn.shape, label_all.shape)

219/219 [==============================] - 0s 813us/step - loss: 1.0786 - accuracy: 0.6883
Final Accuracy on Test Data: 0.6882857084274292
875/875 [==============================] - 1s 616us/step
----->
(28000, 120)
the shape of cnn output features (28000, 120) (28000, 6)


In [6]:
arr=np.array([ [-10, -29, -19, -6, -10, -4, -20, -37, -13, -16, -9, -13, -6, -16, -6, -8, -12, -8, -8, -11, -15, 2, -16, 3, -1, 4, -4, -5, 13, -21, -9, -2, -17, -11, -7, -11, -3, 37, -17, -11, 5, -8, -10, 22, 3, -18, 15, 10, 9, 5, 4, -1, -16, -10, -13, 8, 5, 13, -16, -25, -5, -10, -11, -45]])
arr=scaler.transform(arr)
arr=arr.reshape(-1,1,64,1)
print(cnn_up_to_dropout.predict(arr))

1/1 [==============================] - 0s 42ms/step
[[2.67115012e-02 1.44550586e-02 7.27990866e-01 3.45818400e-02
  1.12410514e-02 6.08581528e-02 7.69591451e-01 5.60903234e-12
  6.87231441e-05 6.05575391e-04 7.59376795e-04 4.23882296e-03
  5.61983138e-03 4.63047036e-05 1.25893741e-03 9.40969959e-03
  7.58736432e-02 2.28849184e-02 2.97748029e-01 8.33069533e-03
  5.01698554e-02 6.09604008e-02 9.25074935e-01 8.98138404e-01
  2.38752980e-02 3.15720625e-02 6.49946034e-02 9.61191836e-04
  8.52282286e-01 4.96179424e-02 5.93166566e-04 1.99558929e-01
  1.74885860e-03 1.05518408e-01 7.38537431e-01 1.28787395e-03
  8.66018161e-02 1.12727121e-03 1.79448718e-04 6.79905725e-06
  1.40208483e-01 1.28719201e-02 3.46054602e-03 2.88400114e-01
  6.10422492e-01 1.15091912e-02 8.85000527e-02 2.66742241e-03
  1.94755737e-02 5.63979521e-02 5.46855889e-02 6.23123109e-01
  1.75464363e-03 1.11886497e-04 1.97564880e-03 9.89826471e-02
  3.04032164e-03 4.58213054e-02 1.06168069e-01 1.65810362e-02
  2.52404839e-01 1

In [7]:

arr=np.array([ [-39, -54, -44, -26, -35, -11, -7, -22, -28, -41, -42, -20, -16, -24, -17, -26, -34, -46, -29, -25, -7, -51, -40, -26, -82, -76, -44, -21, -7, -64, -82, -27, -58, -52, -43, -40, -4, -3, -68, -21, -86, -29, -36, 27, -32, 5, -5, -9, -14, -39, -52, -38, -26, -7, 5, 22, -7, -51, 14, 29, 17, -16, 46, 28]])
arr=scaler.transform(arr)
# print(arr)
arr=arr.reshape(-1,1,64,1)
print(cnn_up_to_dropout.predict(arr))

1/1 [==============================] - 0s 15ms/step
[[1.08886482e-02 7.97193206e-04 7.08246231e-02 9.53957718e-03
  3.66093969e-04 5.69515047e-04 9.88728762e-01 7.08238721e-15
  1.88753358e-04 2.49188347e-03 4.94986816e-05 5.95250726e-02
  2.36386731e-02 1.63399716e-04 9.50682163e-01 1.38300809e-03
  3.47892055e-05 1.83185365e-03 9.60418761e-01 6.62252866e-03
  1.87304448e-02 2.23146509e-02 1.25628114e-01 1.50913065e-02
  8.06185044e-03 1.96043234e-02 2.05606166e-02 8.62595043e-04
  2.49375924e-02 4.93999086e-02 2.03679429e-05 3.76143903e-02
  2.09219381e-01 7.55837634e-02 3.36171359e-01 3.21741074e-01
  8.47429186e-02 1.26924133e-04 4.31488843e-05 2.09298360e-05
  7.81897753e-02 1.52115361e-03 3.41556477e-03 7.73897534e-03
  6.29597111e-04 2.31953361e-03 5.42440638e-02 5.60541230e-05
  1.59652019e-03 6.43028691e-02 1.84399206e-02 7.62903392e-01
  1.35657057e-04 2.66226381e-01 2.98281317e-03 1.43794231e-02
  3.95326177e-04 8.40278808e-03 7.38315433e-02 3.52404802e-03
  2.41749376e-01 9

In [8]:
no_fea = feature_all.shape[-1]
print(no_fea)

final = feature_all.shape[0]  # Assuming 'final' is defined somewhere
feature_all = feature_all.reshape([final, 1, no_fea])
fin=label_all.shape[0]
all_label=label_all
# label_all=label_all.reshape([fin,1,6])
print(tf.argmax(label_all, 1))
print(label_all.shape)


middle_number = 21000  # You should define this
feature_training = feature_all[0:middle_number]
feature_testing = feature_all[middle_number:final]
label_training = label_all[0:middle_number]
label_testing = label_all[middle_number:final]

print(feature_training.shape)
print(feature_testing.shape)

nodes = 264
lameda = 0.004
lr = 0.005
batch_size = final - middle_number

train_fea = []
n_group = 3

for i in range(n_group):
    f = feature_training[(0 + batch_size * i):(batch_size + batch_size * i)]
    train_fea.append(f)
print(train_fea[0].shape)

train_label = []

for i in range(n_group):
    f = label_training[(0 + batch_size * i):(batch_size + batch_size * i), :]
    train_label.append(f)
print(train_label[0].shape)

64
tf.Tensor([1 2 1 ... 3 1 1], shape=(28000,), dtype=int64)
(28000, 6)
(21000, 1, 64)
(7000, 1, 64)
(7000, 1, 64)
(7000, 6)


In [9]:
# import tensorflow as tf
# from tensorflow import keras

# n_inputs = no_fea
# n_steps = 1  # time steps
# nodes=264
# n_hidden1_units = nodes  # neurons in the hidden layer
# n_hidden2_units = nodes
# n_hidden3_units = nodes
# n_hidden4_units = nodes
# n_classes = n_classes

# # Define the model using the Keras API
# weight=tf.keras.initializers.RandomNormal()
# bias=tf.keras.initializers.Constant(0.1)
# rnn_model = keras.Sequential([
#     keras.layers.Input(shape=(n_steps, n_inputs)),  # Input layer
#     keras.layers.Dense(n_hidden1_units,kernel_initializer=weight,bias_initializer=bias),
#     keras.layers.Dense(n_hidden2_units,kernel_initializer=weight,bias_initializer=bias),
#     keras.layers.Dense(n_hidden3_units,kernel_initializer=weight,bias_initializer=bias),
#     keras.layers.Dense(n_hidden4_units,kernel_initializer=weight,bias_initializer=bias),
#     keras.layers.LSTM(n_hidden1_units,return_sequences=True ,use_bias=False),  # LSTM layer 1
#     keras.layers.LSTM(n_hidden2_units,return_sequences=True,use_bias=False),  # LSTM layer 2
#     # keras.layers.LSTM(n_hidden3_units, return_sequences=True),  # LSTM layer 3
#     # keras.layers.LSTM(n_hidden4_units, return_sequences=False),  # LSTM layer 4
#     keras.layers.Dense(n_classes, activation='softmax')  # Output layer
# ])
# # rnn_model.add()
# label_training=label_training.reshape((21000,1,6))
# # Compile the model
# rnn_model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

# # Now, you can train the model using your data with model.fit().

# # Example usage:
# # model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
# # feature_training = np.concatenate(train_fea, axis=0)
# # label_training = np.concatenate(train_label, axis=0)

# rnn_model.fit(feature_training, label_training, batch_size=batch_size, epochs=25, verbose=2)


In [10]:
from tensorflow.keras.losses import Loss
from tensorflow.keras.regularizers import l2
from tensorflow.keras import losses


def rnn_loss(alpha=0.001):
    # Define the custom loss function
    def loss_function(y_true, y_pred):
        # Calculate the softmax cross-entropy loss
        # print(y_pred)
        # print(y_true)
        cross_entropy_loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)

        # cross_entropy_loss=losses.CategoricalCrossentropy()(y_true,y_pred)
        # Calculate the L2 loss
        l2_loss = alpha * tf.reduce_sum([tf.nn.l2_loss(var) for var in rnn_model.trainable_variables])

        # Combine the two losses
        total_loss = cross_entropy_loss + l2_loss

        return total_loss

    return loss_function

In [11]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Reshape
from tensorflow.keras.models import Model
import numpy as np

# Define the model
n_inputs =64
n_steps = 1 
# n_inputs = 10  # Replace with your actual input size
# n_steps = 20   # Replace with your desired sequence length
n_hidden4_units = 264  # Replace with your desired number of hidden units
n_outputs = 6  # Replace with your desired output size
batch_size = 7000  # Replace with your desired batch size

# Create the input layer
inputs = Input(shape=(n_steps, n_inputs))

# Reshape the input
# X = Reshape((-1, n_inputs))(inputs)
bias=tf.keras.initializers.Constant(0.1)
weight=tf.keras.initializers.RandomNormal()
# Hidden layers
X_hidd1 = Dense(n_hidden4_units,kernel_initializer=weight,bias_initializer=bias)(inputs)
X_hidd2 = Dense(n_hidden4_units,kernel_initializer=weight,bias_initializer=bias)(X_hidd1)
X_hidd3 = Dense(n_hidden4_units,kernel_initializer=weight,bias_initializer=bias)(X_hidd2)
X_hidd4 = Dense(n_hidden4_units,kernel_initializer=weight,bias_initializer=bias)(X_hidd3)

# Reshape back to the original shape
X_in = Reshape((n_steps, n_hidden4_units))(X_hidd4)

# Create LSTM cells
lstm_cell_1 = LSTM(n_hidden4_units, return_sequences=True)(X_in)
lstm_cell_2 = LSTM(n_hidden4_units, return_sequences=True)(lstm_cell_1)

# Merge the LSTM cells
merged_lstm_cells = tf.keras.layers.concatenate([lstm_cell_1, lstm_cell_2])
X_hidd5=Dense(n_hidden4_units,kernel_initializer=weight,bias_initializer=bias)(merged_lstm_cells)

# Define the output layer
outputs = Dense(n_outputs)(X_hidd5)
# Create the model
rnn_model = Model(inputs=inputs, outputs=outputs)

# Compile the model
rnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss=rnn_loss(),metrics=['accuracy'])

# Print a summary of the model
rnn_model.summary()

D:\Anaconda\envs\tf2\lib\site-packages\keras\initializers\initializers_v2.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1, 64)]      0           []                               
                                                                                                  
 dense_2 (Dense)                (None, 1, 264)       17160       ['input_1[0][0]']                
                                                                                                  
 dense_3 (Dense)                (None, 1, 264)       69960       ['dense_2[0][0]']                
                                                                                                  
 dense_4 (Dense)                (None, 1, 264)       69960       ['dense_3[0][0]']                
                                                                                            

In [12]:
rnn_model.fit(feature_training, label_training, batch_size=batch_size, epochs=500)

Epoch 1/500
3/3 [==============================] - 3s 284ms/step - loss: 3.1707 - accuracy: 0.1673
Epoch 2/500
3/3 [==============================] - 1s 288ms/step - loss: 2.7885 - accuracy: 0.1638
Epoch 3/500
3/3 [==============================] - 1s 284ms/step - loss: 2.5435 - accuracy: 0.1198
Epoch 4/500
3/3 [==============================] - 1s 289ms/step - loss: 2.5044 - accuracy: 0.0779
Epoch 5/500
3/3 [==============================] - 1s 291ms/step - loss: 2.3914 - accuracy: 0.1384
Epoch 6/500
3/3 [==============================] - 1s 286ms/step - loss: 2.3301 - accuracy: 0.1537
Epoch 7/500
3/3 [==============================] - 1s 289ms/step - loss: 2.2545 - accuracy: 0.1252
Epoch 8/500
3/3 [==============================] - 1s 288ms/step - loss: 2.1882 - accuracy: 0.0932
Epoch 9/500
3/3 [==============================] - 1s 290ms/step - loss: 2.1855 - accuracy: 0.0831
Epoch 10/500
3/3 [==============================] - 1s 286ms/step - loss: 2.0788 - accuracy: 0.0890
Epoch 11/

3/3 [==============================] - 1s 322ms/step - loss: 0.9773 - accuracy: 0.0568
Epoch 84/500
3/3 [==============================] - 1s 310ms/step - loss: 0.9199 - accuracy: 0.0570
Epoch 85/500
3/3 [==============================] - 1s 302ms/step - loss: 0.8487 - accuracy: 0.0548
Epoch 86/500
3/3 [==============================] - 1s 305ms/step - loss: 0.8045 - accuracy: 0.0562
Epoch 87/500
3/3 [==============================] - 1s 297ms/step - loss: 0.7720 - accuracy: 0.0563
Epoch 88/500
3/3 [==============================] - 1s 298ms/step - loss: 0.7281 - accuracy: 0.0559
Epoch 89/500
3/3 [==============================] - 1s 310ms/step - loss: 0.6956 - accuracy: 0.0570
Epoch 90/500
3/3 [==============================] - 1s 293ms/step - loss: 0.6748 - accuracy: 0.0531
Epoch 91/500
3/3 [==============================] - 1s 300ms/step - loss: 0.6853 - accuracy: 0.0591
Epoch 92/500
3/3 [==============================] - 1s 299ms/step - loss: 0.6496 - accuracy: 0.0546
Epoch 93/500


3/3 [==============================] - 1s 354ms/step - loss: 1.5626 - accuracy: 0.0577
Epoch 165/500
3/3 [==============================] - 1s 354ms/step - loss: 1.5144 - accuracy: 0.0616
Epoch 166/500
3/3 [==============================] - 1s 355ms/step - loss: 1.4729 - accuracy: 0.0558
Epoch 167/500
3/3 [==============================] - 1s 349ms/step - loss: 1.4288 - accuracy: 0.0584
Epoch 168/500
3/3 [==============================] - 1s 348ms/step - loss: 1.3921 - accuracy: 0.0614
Epoch 169/500
3/3 [==============================] - 1s 355ms/step - loss: 1.3717 - accuracy: 0.0533
Epoch 170/500
3/3 [==============================] - 1s 351ms/step - loss: 1.3053 - accuracy: 0.0643
Epoch 171/500
3/3 [==============================] - 1s 368ms/step - loss: 1.2666 - accuracy: 0.0527
Epoch 172/500
3/3 [==============================] - 1s 361ms/step - loss: 1.2106 - accuracy: 0.0613
Epoch 173/500
3/3 [==============================] - 1s 351ms/step - loss: 1.1564 - accuracy: 0.0536
Epoc

3/3 [==============================] - 1s 350ms/step - loss: 0.4105 - accuracy: 0.0550
Epoch 246/500
3/3 [==============================] - 1s 351ms/step - loss: 0.5206 - accuracy: 0.0551
Epoch 247/500
3/3 [==============================] - 1s 349ms/step - loss: 0.8736 - accuracy: 0.0557
Epoch 248/500
3/3 [==============================] - 1s 358ms/step - loss: 0.8615 - accuracy: 0.0565
Epoch 249/500
3/3 [==============================] - 1s 356ms/step - loss: 0.7724 - accuracy: 0.0564
Epoch 250/500
3/3 [==============================] - 1s 354ms/step - loss: 0.6689 - accuracy: 0.0539
Epoch 251/500
3/3 [==============================] - 1s 350ms/step - loss: 0.6124 - accuracy: 0.0558
Epoch 252/500
3/3 [==============================] - 1s 364ms/step - loss: 0.5657 - accuracy: 0.0559
Epoch 253/500
3/3 [==============================] - 1s 351ms/step - loss: 0.5276 - accuracy: 0.0557
Epoch 254/500
3/3 [==============================] - 1s 351ms/step - loss: 0.4989 - accuracy: 0.0547
Epoc

3/3 [==============================] - 1s 380ms/step - loss: 0.3556 - accuracy: 0.0553
Epoch 327/500
3/3 [==============================] - 1s 386ms/step - loss: 0.3508 - accuracy: 0.0553
Epoch 328/500
3/3 [==============================] - 1s 386ms/step - loss: 0.3385 - accuracy: 0.0554
Epoch 329/500
3/3 [==============================] - 1s 387ms/step - loss: 0.3299 - accuracy: 0.0556
Epoch 330/500
3/3 [==============================] - 1s 386ms/step - loss: 0.3246 - accuracy: 0.0552
Epoch 331/500
3/3 [==============================] - 1s 381ms/step - loss: 0.3221 - accuracy: 0.0555
Epoch 332/500
3/3 [==============================] - 1s 372ms/step - loss: 0.3214 - accuracy: 0.0548
Epoch 333/500
3/3 [==============================] - 1s 376ms/step - loss: 0.3183 - accuracy: 0.0551
Epoch 334/500
3/3 [==============================] - 1s 369ms/step - loss: 0.3145 - accuracy: 0.0555
Epoch 335/500
3/3 [==============================] - 1s 364ms/step - loss: 0.3122 - accuracy: 0.0550
Epoc

3/3 [==============================] - 1s 349ms/step - loss: 0.8552 - accuracy: 0.0550
Epoch 408/500
3/3 [==============================] - 1s 362ms/step - loss: 0.8181 - accuracy: 0.0554
Epoch 409/500
3/3 [==============================] - 1s 363ms/step - loss: 0.7931 - accuracy: 0.0558
Epoch 410/500
3/3 [==============================] - 1s 356ms/step - loss: 0.8150 - accuracy: 0.0561
Epoch 411/500
3/3 [==============================] - 1s 351ms/step - loss: 0.7746 - accuracy: 0.0561
Epoch 412/500
3/3 [==============================] - 1s 354ms/step - loss: 0.7266 - accuracy: 0.0548
Epoch 413/500
3/3 [==============================] - 1s 354ms/step - loss: 0.7046 - accuracy: 0.0555
Epoch 414/500
3/3 [==============================] - 1s 350ms/step - loss: 0.6651 - accuracy: 0.0556
Epoch 415/500
3/3 [==============================] - 1s 353ms/step - loss: 0.6413 - accuracy: 0.0557
Epoch 416/500
3/3 [==============================] - 1s 352ms/step - loss: 0.6214 - accuracy: 0.0559
Epoc

3/3 [==============================] - 1s 297ms/step - loss: 0.3330 - accuracy: 0.0554
Epoch 489/500
3/3 [==============================] - 1s 295ms/step - loss: 0.3298 - accuracy: 0.0557
Epoch 490/500
3/3 [==============================] - 1s 294ms/step - loss: 0.3278 - accuracy: 0.0554
Epoch 491/500
3/3 [==============================] - 1s 301ms/step - loss: 0.3221 - accuracy: 0.0555
Epoch 492/500
3/3 [==============================] - 1s 297ms/step - loss: 0.3228 - accuracy: 0.0556
Epoch 493/500
3/3 [==============================] - 1s 302ms/step - loss: 0.3219 - accuracy: 0.0556
Epoch 494/500
3/3 [==============================] - 1s 297ms/step - loss: 0.3262 - accuracy: 0.0557
Epoch 495/500
3/3 [==============================] - 1s 316ms/step - loss: 0.3215 - accuracy: 0.0555
Epoch 496/500
3/3 [==============================] - 1s 289ms/step - loss: 0.3163 - accuracy: 0.0550
Epoch 497/500
3/3 [==============================] - 1s 292ms/step - loss: 0.3185 - accuracy: 0.0558
Epoc

In [13]:
rnn_model.evaluate(feature_testing,label_testing)

219/219 [==============================] - 1s 3ms/step - loss: 0.4929 - accuracy: 0.0573


[0.4929251968860626, 0.05728571489453316]

In [14]:
rnn_up_to_dropout = keras.Model(inputs=rnn_model.inputs, outputs=rnn_model.get_layer('lstm_1').output)
feature_0 = rnn_up_to_dropout.predict(train_fea[0])  #initialise

for i in range(1, n_group):
    feature_i = rnn_up_to_dropout.predict(train_fea[i])
    feature_0 = np.vstack((feature_0, feature_i))

# Get predictions up to the hidden4 layer for the testing data
feature_b = rnn_up_to_dropout.predict(feature_testing)

# Concatenate the features from training and testing data
feature_all_rnn = np.vstack((feature_0, feature_b))

# Save the predictions to a file or use them as needed
# ...

# Save the original model for later training
rnn_model.save("rnn_with_dropout.h5")

219/219 [==============================] - 1s 3ms/step


In [15]:
feature_all_rnn.shape

(28000, 1, 264)

In [16]:
arr=np.array([ [-10, -29, -19, -6, -10, -4, -20, -37, -13, -16, -9, -13, -6, -16, -6, -8, -12, -8, -8, -11, -15, 2, -16, 3, -1, 4, -4, -5, 13, -21, -9, -2, -17, -11, -7, -11, -3, 37, -17, -11, 5, -8, -10, 22, 3, -18, 15, 10, 9, 5, 4, -1, -16, -10, -13, 8, 5, 13, -16, -25, -5, -10, -11, -45]])
arr=scaler.transform(arr)
arr=arr.reshape(-1,1,64)
print(rnn_up_to_dropout.predict(arr).shape)

1/1 [==============================] - 0s 17ms/step
(1, 1, 264)


In [17]:
arr=np.array([ [-39, -54, -44, -26, -35, -11, -7, -22, -28, -41, -42, -20, -16, -24, -17, -26, -34, -46, -29, -25, -7, -51, -40, -26, -82, -76, -44, -21, -7, -64, -82, -27, -58, -52, -43, -40, -4, -3, -68, -21, -86, -29, -36, 27, -32, 5, -5, -9, -14, -39, -52, -38, -26, -7, 5, 22, -7, -51, 14, 29, 17, -16, 46, 28]])
arr=scaler.transform(arr)
# print(arr)
arr=arr.reshape(-1,1,64)
print(rnn_up_to_dropout.predict(arr).shape)

1/1 [==============================] - 0s 16ms/step
(1, 1, 264)


In [18]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import numpy as np

# Assuming you have already defined feature_all_rnn, feature_all_cnn, and label_all

# Stack feature_all_rnn and feature_all_cnn horizontally
print(feature_all_cnn.shape)
print(feature_all_rnn.shape)
feature_all_rnn=feature_all_rnn.reshape((-1,264))


feature_all = np.hstack((feature_all_rnn, feature_all_cnn))
print(feature_all.shape)
no_fea = feature_all.shape[-1]  #384

middle_number = 21000  # Define middle_number and final
final = len(feature_all)

feature_training = feature_all[0:middle_number]
feature_testing = feature_all[middle_number:final]
label_training = label_all[0:middle_number]

train_fea = feature_training

group = 3
display_step = 10
training_epochs = 100

# Network Parameters
n_hidden_1 = 800  # 1st layer num features, should be times of 8
n_hidden_2 = 100
n_input_ae = no_fea  #384
print("NO_FEA",no_fea)
# Create an input layer
X = Input(shape=(n_input_ae,))

bias=tf.keras.initializers.RandomNormal()
weight=tf.keras.initializers.RandomNormal()

# Define the encoder layers
encoder_h1 = Dense(n_hidden_1, activation='sigmoid',kernel_initializer=weight,bias_initializer=bias)(X)
# encoder_h2 = Dense(n_hidden_2, activation='sigmoid')(encoder_h1)

# Define the decoder layers
# decoder_h1 = Dense(n_hidden_1, activation='sigmoid')(encoder_h2)
decoder_h2 = Dense(n_input_ae, activation='sigmoid',kernel_initializer=weight,bias_initializer=bias)(encoder_h1)

# Create the autoencoder model
autoencoder = Model(inputs=X, outputs=decoder_h2)

# Compile the model
autoencoder.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.01), loss='mean_squared_error')


autoencoder.fit(train_fea, train_fea, epochs=200, batch_size=7000, verbose=2)


print(autoencoder.summary())
print("Optimization Finished!")

# You can access the encoder output like this:
encoder_model = Model(inputs=X, outputs=encoder_h1)
a = encoder_model.predict(train_fea)
b=encoder_model.predict(feature_testing)

# Save the model
autoencoder.save('autoencoder.h5')

(28000, 120)
(28000, 1, 264)
(28000, 384)
NO_FEA 384
Epoch 1/200


D:\Anaconda\envs\tf2\lib\site-packages\keras\initializers\initializers_v2.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


3/3 - 0s - loss: 0.1026 - 450ms/epoch - 150ms/step
Epoch 2/200
3/3 - 0s - loss: 0.0161 - 243ms/epoch - 81ms/step
Epoch 3/200
3/3 - 0s - loss: 0.0157 - 250ms/epoch - 83ms/step
Epoch 4/200
3/3 - 0s - loss: 0.0154 - 244ms/epoch - 81ms/step
Epoch 5/200
3/3 - 0s - loss: 0.0152 - 251ms/epoch - 84ms/step
Epoch 6/200
3/3 - 0s - loss: 0.0151 - 245ms/epoch - 82ms/step
Epoch 7/200
3/3 - 0s - loss: 0.0149 - 247ms/epoch - 82ms/step
Epoch 8/200
3/3 - 0s - loss: 0.0148 - 249ms/epoch - 83ms/step
Epoch 9/200
3/3 - 0s - loss: 0.0146 - 254ms/epoch - 85ms/step
Epoch 10/200
3/3 - 0s - loss: 0.0145 - 250ms/epoch - 83ms/step
Epoch 11/200
3/3 - 0s - loss: 0.0143 - 251ms/epoch - 84ms/step
Epoch 12/200
3/3 - 0s - loss: 0.0141 - 247ms/epoch - 82ms/step
Epoch 13/200
3/3 - 0s - loss: 0.0140 - 246ms/epoch - 82ms/step
Epoch 14/200
3/3 - 0s - loss: 0.0138 - 247ms/epoch - 82ms/step
Epoch 15/200
3/3 - 0s - loss: 0.0137 - 248ms/epoch - 83ms/step
Epoch 16/200
3/3 - 0s - loss: 0.0135 - 249ms/epoch - 83ms/step
Epoch 17/200

Epoch 131/200
3/3 - 0s - loss: 0.0073 - 264ms/epoch - 88ms/step
Epoch 132/200
3/3 - 0s - loss: 0.0073 - 261ms/epoch - 87ms/step
Epoch 133/200
3/3 - 0s - loss: 0.0072 - 253ms/epoch - 84ms/step
Epoch 134/200
3/3 - 0s - loss: 0.0072 - 254ms/epoch - 85ms/step
Epoch 135/200
3/3 - 0s - loss: 0.0072 - 257ms/epoch - 86ms/step
Epoch 136/200
3/3 - 0s - loss: 0.0072 - 267ms/epoch - 89ms/step
Epoch 137/200
3/3 - 0s - loss: 0.0072 - 260ms/epoch - 87ms/step
Epoch 138/200
3/3 - 0s - loss: 0.0072 - 249ms/epoch - 83ms/step
Epoch 139/200
3/3 - 0s - loss: 0.0071 - 246ms/epoch - 82ms/step
Epoch 140/200
3/3 - 0s - loss: 0.0071 - 252ms/epoch - 84ms/step
Epoch 141/200
3/3 - 0s - loss: 0.0071 - 248ms/epoch - 83ms/step
Epoch 142/200
3/3 - 0s - loss: 0.0071 - 271ms/epoch - 90ms/step
Epoch 143/200
3/3 - 0s - loss: 0.0071 - 247ms/epoch - 82ms/step
Epoch 144/200
3/3 - 0s - loss: 0.0071 - 247ms/epoch - 82ms/step
Epoch 145/200
3/3 - 0s - loss: 0.0070 - 246ms/epoch - 82ms/step
Epoch 146/200
3/3 - 0s - loss: 0.0070 - 

In [19]:
fea_train=train_fea[0].reshape((1,384))
store=autoencoder.predict(fea_train)
print(store)

1/1 [==============================] - 0s 33ms/step
[[6.86272746e-03 4.15187655e-03 5.79268392e-03 3.96649912e-03
  4.97839646e-03 4.82838019e-04 9.01975483e-03 3.61755164e-03
  4.13777633e-03 2.85499962e-03 4.24294686e-03 4.69666021e-03
  3.66288214e-03 3.82860634e-03 4.38177073e-03 4.72481130e-03
  6.18357724e-03 5.44882286e-03 4.72815847e-03 6.47916831e-03
  5.93007402e-03 3.77183571e-03 2.74221562e-02 2.57925619e-03
  7.25650648e-03 4.46847780e-03 4.21020295e-03 5.37785050e-03
  5.23720914e-03 6.21552253e-03 4.40538395e-03 6.59673940e-03
  3.49907996e-03 4.20081709e-03 3.04715987e-03 3.92799592e-03
  5.17694233e-03 4.17626649e-03 3.85576626e-03 4.38370183e-03
  6.11870317e-03 4.77530388e-03 4.56588669e-03 4.01936937e-03
  3.92905949e-03 4.69836779e-03 5.04054548e-03 3.56000313e-03
  8.13915115e-03 3.76331806e-03 4.36946005e-03 3.19684879e-03
  3.92153068e-03 2.23372248e-03 3.96213681e-03 5.57881687e-03
  3.35452752e-03 5.87099139e-03 3.54661467e-03 5.50159859e-03
  4.50673699e-03 3

In [20]:
##XGBoost
import xgboost as xgb
import time
import os

test_label=all_label[middle_number:final]
train_label=all_label[0:middle_number]

xg_train = xgb.DMatrix(a, label=np.argmax(train_label,1))
xg_test = xgb.DMatrix(b, label=np.argmax(test_label,1))

# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softprob' # can I replace softmax by SVM??
# softprob produce a matrix with probability value of each class
# scale weight of positive examples
param['eta'] = 0.5
param['metrics']=''
param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['subsample']=0.9
# param['lambda']=1
param['num_class'] =n_classes



np.set_printoptions(threshold=np.inf, linewidth=np.nan)
watchlist = [ (xg_train,'train'), (xg_test, 'test') ]
num_round = 500
bst = xgb.train(param, xg_train, num_round, watchlist );

pred = bst.predict( xg_test );


xgb_model_path = os.path.join('./', "xgb_model.model")
bst.save_model(xgb_model_path)
# print("XGBoost model saved at:", xgb_model_path)

D:\Anaconda\envs\tf2\lib\site-packages\xgboost\core.py:571: FutureWarning: Pass `evals` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  format(", ".join(args_msg)), FutureWarning


[11:32:03] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "metrics", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-mlogloss:1.12284	test-mlogloss:1.20572
[1]	train-mlogloss:0.84843	test-mlogloss:0.97612
[2]	train-mlogloss:0.67507	test-mlogloss:0.82994
[3]	train-mlogloss:0.54626	test-mlogloss:0.72059
[4]	train-mlogloss:0.45242	test-mlogloss:0.64279
[5]	train-mlogloss:0.38070	test-mlogloss:0.58331
[6]	train-mlogloss:0.32372	test-mlogloss:0.53514
[7]	train-mlogloss:0.27968	test-mlogloss:0.49783
[8]	train-mlogloss:0.24344	test-mlogloss:0.46688
[9]	train-mlogloss:0.21202	test-mlogloss:0.43844
[10]	train-mlogloss:0.18455	test-mlogloss:0.41558
[11]	train-mlogloss:0.16276	test-m

[155]	train-mlogloss:0.01015	test-mlogloss:0.22369
[156]	train-mlogloss:0.01015	test-mlogloss:0.22377
[157]	train-mlogloss:0.01014	test-mlogloss:0.22375
[158]	train-mlogloss:0.01014	test-mlogloss:0.22377
[159]	train-mlogloss:0.01014	test-mlogloss:0.22380
[160]	train-mlogloss:0.01013	test-mlogloss:0.22364
[161]	train-mlogloss:0.01013	test-mlogloss:0.22377
[162]	train-mlogloss:0.01013	test-mlogloss:0.22370
[163]	train-mlogloss:0.01012	test-mlogloss:0.22383
[164]	train-mlogloss:0.01012	test-mlogloss:0.22358
[165]	train-mlogloss:0.01011	test-mlogloss:0.22387
[166]	train-mlogloss:0.01011	test-mlogloss:0.22393
[167]	train-mlogloss:0.01011	test-mlogloss:0.22401
[168]	train-mlogloss:0.01010	test-mlogloss:0.22380
[169]	train-mlogloss:0.01010	test-mlogloss:0.22389
[170]	train-mlogloss:0.01009	test-mlogloss:0.22378
[171]	train-mlogloss:0.01009	test-mlogloss:0.22383
[172]	train-mlogloss:0.01009	test-mlogloss:0.22366
[173]	train-mlogloss:0.01009	test-mlogloss:0.22364
[174]	train-mlogloss:0.01008	te

[316]	train-mlogloss:0.00988	test-mlogloss:0.22557
[317]	train-mlogloss:0.00988	test-mlogloss:0.22561
[318]	train-mlogloss:0.00988	test-mlogloss:0.22553
[319]	train-mlogloss:0.00988	test-mlogloss:0.22560
[320]	train-mlogloss:0.00988	test-mlogloss:0.22575
[321]	train-mlogloss:0.00988	test-mlogloss:0.22567
[322]	train-mlogloss:0.00988	test-mlogloss:0.22571
[323]	train-mlogloss:0.00988	test-mlogloss:0.22578
[324]	train-mlogloss:0.00988	test-mlogloss:0.22590
[325]	train-mlogloss:0.00988	test-mlogloss:0.22576
[326]	train-mlogloss:0.00988	test-mlogloss:0.22579
[327]	train-mlogloss:0.00988	test-mlogloss:0.22580
[328]	train-mlogloss:0.00988	test-mlogloss:0.22577
[329]	train-mlogloss:0.00987	test-mlogloss:0.22577
[330]	train-mlogloss:0.00987	test-mlogloss:0.22579
[331]	train-mlogloss:0.00987	test-mlogloss:0.22586
[332]	train-mlogloss:0.00987	test-mlogloss:0.22593
[333]	train-mlogloss:0.00987	test-mlogloss:0.22590
[334]	train-mlogloss:0.00987	test-mlogloss:0.22586
[335]	train-mlogloss:0.00987	te

[477]	train-mlogloss:0.00982	test-mlogloss:0.22841
[478]	train-mlogloss:0.00982	test-mlogloss:0.22838
[479]	train-mlogloss:0.00982	test-mlogloss:0.22838
[480]	train-mlogloss:0.00982	test-mlogloss:0.22842
[481]	train-mlogloss:0.00982	test-mlogloss:0.22844
[482]	train-mlogloss:0.00982	test-mlogloss:0.22846
[483]	train-mlogloss:0.00982	test-mlogloss:0.22845
[484]	train-mlogloss:0.00982	test-mlogloss:0.22839
[485]	train-mlogloss:0.00982	test-mlogloss:0.22835
[486]	train-mlogloss:0.00982	test-mlogloss:0.22830
[487]	train-mlogloss:0.00981	test-mlogloss:0.22830
[488]	train-mlogloss:0.00981	test-mlogloss:0.22831
[489]	train-mlogloss:0.00981	test-mlogloss:0.22833
[490]	train-mlogloss:0.00981	test-mlogloss:0.22841
[491]	train-mlogloss:0.00981	test-mlogloss:0.22844
[492]	train-mlogloss:0.00981	test-mlogloss:0.22847
[493]	train-mlogloss:0.00981	test-mlogloss:0.22848
[494]	train-mlogloss:0.00981	test-mlogloss:0.22847
[495]	train-mlogloss:0.00981	test-mlogloss:0.22849
[496]	train-mlogloss:0.00981	te

In [21]:
b.shape

(7000, 800)

In [22]:
test_label.shape

(7000, 6)

In [23]:
pred.shape

(7000, 6)

In [24]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(np.argmax(test_label,1), np.argmax(pred,1))
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 93.29%


In [25]:
import joblib
joblib.dump(scaler, 'std_scaler.bin', compress=True)

['std_scaler.bin']